In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
from multiprocessing import Pool
import time,gc

In [2]:
# f10:上一天该时刻数据的可信度
# f25:上一天该时刻最多进站站台
    
# f11:该站台进站口个数+出站口个数
# f13:站台的邻居个数
    3
# f14:上一天乘客该时刻刷卡类型
# f15:上一天乘客该时间段刷卡类型
# f16:上一天乘客当天刷卡类型(刷卡类型归一化)
    
# f21:上一天非周期用户的数量(一天只有两次刷卡)
# f22:上一天该站台最常见出站台
# f23:上一天该站台全部站出站台人数
# f26:上一天该时刻进站最多目标出站站台(f23的max和std)
# f24:上一天该站台全部站进站台人数
# 分时刻重复上述特征
    
# # 进出站交互特征
# f17:上一天该时刻进出站总人数
# f18:上一天该时间段进出站总人数
# f19:上一天该站总进出流量

# f20:上一天天气情况

3

In [3]:
# 统计进出站每天的变化情况
def get_daily_activity(i):
    if i<10:
        s = '0'+str(i)
    else:
        s = str(i)
    day_activ = pd.read_csv('data/Metro_train/record_2019-01-%s.csv'%s)
    day_activ['time'] = pd.to_datetime(day_activ.time)
    day_activ['stationID'] = day_activ['stationID'].astype('int8')
    day_activ['deviceID'] = day_activ['deviceID'].astype('int32')
    day_activ['status'] = day_activ['status'].astype('int8')
    day_activ['payType'] = day_activ['payType'].astype('int8')
    day_activ['lineID'] = day_activ['lineID'].apply(lambda x: {'A':0,'B':2,'C':1}[x]).astype('int8')
    return day_activ

def construct_df():
    final = pd.DataFrame()
    for i in range(81):
        temp = pd.DataFrame()
        temp['minute'] = list(range(1440))
        temp['stationID'] = i
        final = final.append(temp)
    final['minute'] = final['minute']/10
    final['minute'] = final['minute'].astype('float32')
    final['stationID'] = final['stationID'].astype('int8')
    return final

def get_cum_feature(function,n):
    pool = Pool(8)  #创建拥有5个进程数量的进程池
    timestep = [(x,n) for x in range(1440)]
    count_result =pool.map(function, timestep) 
    pool.close()#关闭进程池，不再接受新的进程
    pool.join()#主进程阻塞等待子进程的退出
    return count_result

# 分站点统计
def median_cum_bystation(params):
    i,n = params
    time_range = [i/10+x for x in range(0-n,1+n)]
    time_range = [143+1+x if x<0 else x for x in time_range]
    station_num = []
    for s in range(80):
        count = []
        for t in time_range:
            try:
                count.append(come_bystation.loc[(s,t),'count_10min'])
            except:
                count.append(0)
        station_num.append([s,i/10,np.median(count)])
    return station_num

In [4]:
# 构建邻接矩阵
roadmap = pd.read_csv('data/Metro_roadMap.csv',index_col=0)
near_node = {}
for index,row in roadmap.iterrows():
    for i in range(80):
        if row[i]==1:
            if index not in near_node:
                near_node[index] = []
            near_node[index].append(i)

In [7]:
t1 = time.time()
all_df = pd.DataFrame()
for date in range(7,26):
    final = construct_df()
    day_activ = get_daily_activity(date)[['time','lineID','stationID','deviceID','status','payType']]
    df = pd.DataFrame()
    for i in range(10):
        day_activ['minute'] = (day_activ['time'].dt.minute-i+day_activ['time'].dt.hour*60)//10+0.1*i
        day_activ['minute'] = day_activ['minute'].apply(lambda x:143+1+x if x<0 else x)
        df = df.append(day_activ)
    df.sort_values(by='minute',inplace=True)
    come = df[df.status==1]
    # 周围半小时中位数
    come_bystation = come.groupby(['stationID','minute']).agg({'status':'count'}).rename(columns={'status':'count_10min'})
    count_result = get_cum_feature(median_cum_bystation,1)
    result = []
    for x in count_result:
        for i in x:
            result.append(i)
    f1 = pd.DataFrame(result)
    f1.rename(columns={0:'stationID',1:'minute',2:'median_30min_bystation'},inplace=True)
    final = final.merge(f1,on=['stationID','minute'],how='left')
    # 上一天该时刻最多进站展台
    come_bystation = come.groupby(['minute','stationID'],as_index=False).agg({'status':'count'}).rename(columns={'status':'count_10min'})
    f2 = come_bystation.sort_values(['minute','count_10min'],ascending=False).\
    drop_duplicates(['minute']).rename(columns={'count_10min':'count_10_max','stationID':'max_count_ID'})
    final = final.merge(f2,on='minute',how='left')
    # 该站台进站口个数+出站口个数
    f3 = come.groupby(['stationID'],as_index=False).agg({'deviceID':'nunique'}).rename(columns={'deviceID':'device_num'})    # f13:站台的邻居个数
    final = final.merge(f3,on='stationID',how='left')
    # 该站台的邻居个数
    final['near_num'] = final['stationID'].apply(lambda x:len(near_node[x]))
    # 刷卡类型处理，该时刻，全天，分站台处理
    count_type = come.groupby(['stationID','minute','payType'],as_index=False).agg({'status':'count'})
    come_bystation = come.groupby(['stationID','minute']).agg({'status':'count'}).rename(columns={'status':'count_10min'})
    rate = []
    for index,row in tqdm(count_type.iterrows()):
        rate.append(row['status']/come_bystation.loc[(row['stationID'],row['minute']),'count_10min'])
    count_type['rate'] = rate
    f4 = count_type[count_type.payType==0].rename(columns={'rate':'type_0_bystatiommin'})[['stationID','minute','type_0_bystatiommin']]
    final = final.merge(f4,on=['stationID','minute'],how='left')
    f5 = count_type[count_type.payType==1].rename(columns={'rate':'type_1_bystatiommin'})[['stationID','minute','type_1_bystatiommin']]
    final = final.merge(f5,on=['stationID','minute'],how='left')
    f6 = count_type[count_type.payType==2].rename(columns={'rate':'type_2_bystatiommin'})[['stationID','minute','type_2_bystatiommin']]
    final = final.merge(f6,on=['stationID','minute'],how='left')
    f7 = count_type[count_type.payType==3].rename(columns={'rate':'type_3_bystatiommin'})[['stationID','minute','type_3_bystatiommin']]
    final = final.merge(f6,on=['stationID','minute'],how='left')
    # 站台，全天
    count_type = come.groupby(['stationID','payType'],as_index=False).agg({'status':'count'})
    come_bystation = come.groupby(['stationID']).agg({'status':'count'}).rename(columns={'status':'count_10min'})
    rate = []
    for index,row in tqdm(count_type.iterrows()):
        rate.append(row['status']/come_bystation.loc[row['stationID'],'count_10min'])
    count_type['rate'] = rate
    for i in range(4):
        f = count_type[count_type.payType==i].rename(columns={'rate':'type_%s_bystatiom'%i})[['stationID','type_%s_bystatiom'%i]]
        final = final.merge(f,on=['stationID'],how='left')
    # 分钟，全天
    count_type = come.groupby(['minute','payType'],as_index=False).agg({'status':'count'})
    come_bystation = come.groupby(['minute']).agg({'status':'count'}).rename(columns={'status':'count_10min'})
    rate = []
    for index,row in tqdm(count_type.iterrows()):
        rate.append(row['status']/come_bystation.loc[row['minute'],'count_10min'])
    count_type['rate'] = rate
    for i in range(4):
        f = count_type[count_type.payType==i].rename(columns={'rate':'type_%s_bymin'%i})[['minute','type_%s_bymin'%i]]
        final = final.merge(f,on=['minute'],how='left')
    t2 = time.time()
    # 数据格式减少内存
    final['date'] = date
    final['date'] = final['date'].astype('int8')
    final.fillna(0,inplace=True)
    for c in final.dtypes.index:
        if final.dtypes[c] == 'int64':
            final[c] = final[c].astype('int32')
    all_df = all_df.append(final)
    print('epoch %d, now f12 %f 秒'%(date,t2-t1))
    #def final
    gc.collect()
                                                     
# progress2缺少时间-所有站台这个维度

328771it [01:51, 2950.64it/s]
320it [00:00, 10615.80it/s]
4704it [00:00, 13675.67it/s]


epoch 7, now f12 151.321766 秒


329831it [01:51, 2970.68it/s]
320it [00:00, 10629.93it/s]
4734it [00:00, 13189.64it/s]


epoch 8, now f12 302.785877 秒


329395it [01:50, 2988.02it/s]
320it [00:00, 10753.16it/s]
4746it [00:00, 13721.59it/s]


...

epoch 23, now f12 2562.418475 秒


334175it [01:51, 2985.35it/s]
320it [00:00, 10814.77it/s]
4789it [00:00, 13736.81it/s]


epoch 24, now f12 2716.962469 秒


335870it [01:51, 3006.49it/s]
320it [00:00, 10888.11it/s]
4763it [00:00, 13562.88it/s]


epoch 25, now f12 2871.953209 秒


In [8]:
all_df.to_pickle('features/progress3.pkl')

In [10]:
all_df.shape

(2216160, 20)